## BTC and its Futures

We illustrate how 4 quantities change with respect to each other in time, namely:

1. BTC price

2. BTC Futures price

3. median spread of futures price

4. number of trades on the future

We can see the spread is more volatile father from expiration, ultimately hovering around the minimum price.
Further, the number of trades on the future increases drastically as the expiration approaches.  Finally, the futures price largely matches that of the underlying bitcoin price.

In [12]:
import pandas as pd
import numpy as np
import calendar
from datetime import datetime

import matplotlib.pyplot as plt
import mpld3


In [13]:
def btc_cme_expiration_time(tkr, timezone='UTC'):    
    assert tkr[:3] == 'BTC', f'{tkr} is not a CME bitcoin future'    
    d_codes = {'F':1,'G':2,'H':3,'J':4,'K':5,'M':6,'N':7,'Q':8,
               'U':9,'V':10,'X':11,'Z':12}
    d_year = {'9':2019,'0':2020}
    assert tkr[-2] in d_codes, f'Wrong ticker name format {tkr} for month code'
    assert tkr[-1] in d_year, f'Wrong ticker name format {tkr} for year code'    
    year = int(d_year[tkr[-1]])
    month = int(d_codes[tkr[-2]])
    # Last friday of the month
    day = max(week[calendar.FRIDAY] for week in calendar.monthcalendar(year, month))    
    t = pd.to_datetime(f'{year}-{month}-{day} 16:00:00')    
    return t.tz_localize('Europe/London').tz_convert(timezone)

# futures top of book

def fut_day_level(tkr):
    data_path = "../data_dumps"
    curncy_data = f"{data_path}/{tkr}_Curncy_top.csv"
    expiry = btc_cme_expiration_time(tkr)

    df = pd.read_csv(curncy_data, parse_dates=["time"])
    df = df.set_index('time')
    df.index = df.index.tz_localize('UTC')
    df["spread"] = np.abs(df["ask_price"] - df["bid_price"])
    df["days_to_expiry"] = (df.index - expiry).days

    # group futures at the day level to investigate spread
    days_expiry = df[["days_to_expiry"]].groupby(pd.Grouper(freq="D")).min().reset_index().fillna(method="ffill")
    day_trade = df[["spread"]].groupby(pd.Grouper(freq="D")).quantile([0.05, 0.25, 0.5, 0.75, 0.95]).unstack().reset_index().fillna(method="ffill")
    day_price = df["trade_price"].resample("D").median().reset_index().fillna(method="ffill")
    day_counts = df["spread"].resample("D").count().reset_index()

    day_trade["counts"] = day_counts["spread"]
    day_trade["days_to_expiry"] = days_expiry["days_to_expiry"]
    day_trade["fut_price"] = day_price["trade_price"]
    day_trade["expiry"] = tkr
    
    
    return day_trade

In [14]:
# grab futures data for which we have price data

may_fut = fut_day_level('BTCK0')
june_fut = fut_day_level('BTCM0')
july_fut = fut_day_level("BTCN0")


# grab raw bitcoin at the day level

may_btc_prices = pd.read_csv("../data_dumps/tob_05.csv", parse_dates=["time"]).set_index('time')
june_btc_prices = pd.read_csv("../data_dumps/tob_06.csv", parse_dates=["time"]).set_index('time')
july_btc_prices = pd.read_csv("../data_dumps/tob_07.csv", parse_dates=["time"]).set_index('time')
btc = pd.concat([may_btc_prices, june_btc_prices, july_btc_prices], axis=0).reset_index()


In [15]:
# sql-style join futures with raw price data
agg_columns = "time,idx,bid_prc,bid_qty,ask_prc,ask_qty,trd_prc,trd_qty,spread_5,spread_25,spread_50,spread_75,spread_95,counts,days_to_expiry,fut_prc,expiry".split(',')

may_both = pd.merge(btc, may_fut, how='inner', on='time') 
may_both.columns = agg_columns

june_both = pd.merge(btc, june_fut, how='inner', on='time') 
june_both.columns = agg_columns

jul_both = pd.merge(btc, july_fut, how='inner', on='time') 
jul_both.columns = agg_columns


/home/harlan/.local/lib/python3.7/site-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/home/harlan/.local/lib/python3.7/site-packages/pandas/core/generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [16]:
# relevant metrics

fig, axs = plt.subplots(4, 1, figsize=(12,14), sharex='col')
axs[0].plot(jul_both["time"], jul_both["trd_prc"], "k") 
axs[0].set_title("BTC trade price")

axs[1].plot(may_both["time"], may_both["fut_prc"], label="May Expiry") 
axs[1].plot(june_both["time"], june_both["fut_prc"], label="June Expiry") 
axs[1].plot(jul_both["time"], jul_both["fut_prc"], label="July Expiry") 
axs[1].set_title("futures trade price")
axs[1].legend(loc="upper right")

axs[2].plot(may_both["time"], may_both['spread_50']) 
axs[2].plot(june_both["time"], june_both['spread_50'])
axs[2].plot(jul_both["time"], jul_both['spread_50'])
axs[2].set_title("median futures spread")

axs[3].plot(may_both["time"], may_both["counts"]) 
axs[3].plot(june_both["time"], june_both["counts"]) 
axs[3].plot(jul_both["time"], jul_both["counts"]) 
axs[3].set_title("number of futures contracts")

for i in range(0,4):
    axs[i].grid(color='lightgray', alpha=0.5)
    axs[i].set_facecolor("whitesmoke")

mpld3.display(fig)


/home/harlan/.local/lib/python3.7/site-packages/pandas/plotting/_matplotlib/converter.py:256: MatplotlibDeprecationWarning: 
The epoch2num function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
  base = dates.epoch2num(dt.asi8 / 1.0e9)
/home/harlan/.local/lib/python3.7/site-packages/pandas/plotting/_matplotlib/converter.py:256: MatplotlibDeprecationWarning: 
The epoch2num function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
  base = dates.epoch2num(dt.asi8 / 1.0e9)
/home/harlan/.local/lib/python3.7/site-packages/pandas/plotting/_matplotlib/converter.py:256: MatplotlibDeprecationWarning: 
The epoch2num function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
  base = dates.epoch2num(dt.asi8 / 1.0e9)
/home/harlan/.local/lib/python3.7/site-packages/pandas/plotting/_matplotlib/converter.py:256: MatplotlibDeprecationWarning: 
The epoch2num function was deprecated in Matplotlib 3.3 and will be 